In [ ]:
# The code was removed by Watson Studio for sharing.

# Week 5 Assignment: Battle of the Neighbourhoods

## Part 1- Setting up the working environment

In [3]:
!conda install -c conda-forge folium --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    folium-0.9.0               |             py_0          59 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will be IN

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from geopy.geocoders import Nominatim

## Part 2- Importing and cleaning the data

### I will first import the division  into wards of the city of London and store it as a DataFrame

In [1]:
boroughs_url = "https://en.wikipedia.org/wiki/List_of_areas_of_London"

In [5]:
b_website = requests.get(boroughs_url)

In [6]:
b_data = b_website.text

In [7]:
b_soup = BeautifulSoup(b_data, "html.parser")

In [8]:
b_website_table = b_soup.find_all("table", class_="wikitable")[0]
b_website_table

<table class="wikitable sortable" style="clear:both;">
<tbody><tr>
<th>Location</th>
<th>London borough</th>
<th>Post town</th>
<th>Postcode district</th>
<th>Dial code</th>
<th>OS grid ref
</th></tr>
<tr>
<td><a href="/wiki/Abbey_Wood" title="Abbey Wood">Abbey Wood</a></td>
<td>Bexley,  Greenwich <sup class="reference" id="cite_ref-mills1_1-0"><a href="#cite_note-mills1-1">[1]</a></sup></td>
<td>LONDON</td>
<td>SE2</td>
<td>020</td>
<td><span class="plainlinks nourlexpansion" style="white-space: nowrap"><a class="external text" href="https://tools.wmflabs.org/os/coor_g/?pagename=List_of_areas_of_London&amp;params=TQ465785_region%3AGB_scale%3A25000">TQ465785</a></span>
</td></tr>
<tr>
<td><a href="/wiki/Acton,_London" title="Acton, London">Acton</a></td>
<td>Ealing, Hammersmith and Fulham<sup class="reference" id="cite_ref-mills2_2-0"><a href="#cite_note-mills2-2">[2]</a></sup></td>
<td>LONDON</td>
<td>W3, W4</td>
<td>020</td>
<td><span class="plainlinks nourlexpansion" style="white-sp

In [9]:
b_table_rows = b_website_table.find_all("tr")
b_table_rows[:5]

[<tr>
 <th>Location</th>
 <th>London borough</th>
 <th>Post town</th>
 <th>Postcode district</th>
 <th>Dial code</th>
 <th>OS grid ref
 </th></tr>, <tr>
 <td><a href="/wiki/Abbey_Wood" title="Abbey Wood">Abbey Wood</a></td>
 <td>Bexley,  Greenwich <sup class="reference" id="cite_ref-mills1_1-0"><a href="#cite_note-mills1-1">[1]</a></sup></td>
 <td>LONDON</td>
 <td>SE2</td>
 <td>020</td>
 <td><span class="plainlinks nourlexpansion" style="white-space: nowrap"><a class="external text" href="https://tools.wmflabs.org/os/coor_g/?pagename=List_of_areas_of_London&amp;params=TQ465785_region%3AGB_scale%3A25000">TQ465785</a></span>
 </td></tr>, <tr>
 <td><a href="/wiki/Acton,_London" title="Acton, London">Acton</a></td>
 <td>Ealing, Hammersmith and Fulham<sup class="reference" id="cite_ref-mills2_2-0"><a href="#cite_note-mills2-2">[2]</a></sup></td>
 <td>LONDON</td>
 <td>W3, W4</td>
 <td>020</td>
 <td><span class="plainlinks nourlexpansion" style="white-space: nowrap"><a class="external text" h

#### The second column in the table contains numbers in brackets such as [2]. This function will remove them

In [10]:
def remove_nums(borough):
    new_word = ""
    for c in range(len(borough)):
        if borough[c].isalpha():
            new_word += borough[c]
        if borough[c] == " ":
            if c != len(borough)-1 and borough[c+1].isalpha():
                new_word += " "
    return new_word

In [11]:
b_result = []
for tr in b_table_rows:
    td = tr.find_all("td")
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        row[1] = remove_nums(row[1])
        b_result.append(row)
b_result

[['Abbey Wood', 'Bexley Greenwich', 'LONDON', 'SE2', '020', 'TQ465785'],
 ['Acton',
  'Ealing Hammersmith and Fulham',
  'LONDON',
  'W3, W4',
  '020',
  'TQ205805'],
 ['Addington', 'Croydon', 'CROYDON', 'CR0', '020', 'TQ375645'],
 ['Addiscombe', 'Croydon', 'CROYDON', 'CR0', '020', 'TQ345665'],
 ['Albany Park', 'Bexley', 'BEXLEY, SIDCUP', 'DA5, DA14', '020', 'TQ478728'],
 ['Aldborough Hatch', 'Redbridge', 'ILFORD', 'IG2', '020', 'TQ455895'],
 ['Aldgate', 'City', 'LONDON', 'EC3', '020', 'TQ334813'],
 ['Aldwych', 'Westminster', 'LONDON', 'WC2', '020', 'TQ307810'],
 ['Alperton', 'Brent', 'WEMBLEY', 'HA0', '020', 'TQ185835'],
 ['Anerley', 'Bromley', 'LONDON', 'SE20', '020', 'TQ345695'],
 ['Angel', 'Islington', 'LONDON', 'EC1, N1', '020', 'TQ345665'],
 ['Aperfield', 'Bromley', 'WESTERHAM', 'TN16', '01959', 'TQ425585'],
 ['Archway', 'Islington', 'LONDON', 'N19', '020', 'TQ285875'],
 ['Ardleigh Green', 'Havering', 'HORNCHURCH', 'RM11', '01708', 'TQ535895'],
 ['Arkley', 'Barnet', 'BARNET, LOND

#### I will now convert it to a DataFrame.

In [12]:
london_df = pd.DataFrame(b_result, columns=["Ward", "Borough", "Post town", "Postcode district", "Dial code", "OS grid ref"])

In [13]:
london_df.head()

,Ward,Borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,Bexley Greenwich,LONDON,SE2,020,TQ465785
1,Acton,Ealing Hammersmith and Fulham,LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon,CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon,CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


#### I will drop all the columns except the first one as they do not provide useful insight for my study.

In [14]:
london_df.drop(["Post town", "Postcode district", "Dial code", "OS grid ref"], axis=1, inplace=True)
london_df.head(10)

,Ward,Borough
0,Abbey Wood,Bexley Greenwich
1,Acton,Ealing Hammersmith and Fulham
2,Addington,Croydon
3,Addiscombe,Croydon
4,Albany Park,Bexley
5,Aldborough Hatch,Redbridge
6,Aldgate,City
7,Aldwych,Westminster
8,Alperton,Brent
9,Anerley,Bromley


#### The column has no missing values, so I do not need to work with NaN

#### I will look at the number of wards left

In [15]:
london_df.shape

(533, 2)

#### I will now find the latitude and longitude of London for future maps

In [16]:
ldn_address = "London"
geolocator = Nominatim(user_agent = "london-explorer")
location = geolocator.geocode(ldn_address)
london_coordinates = [location.latitude, location.longitude]
london_coordinates

[51.5073219, -0.1276474]

#### I will also use GeoPy to find the latitude and longitude of each ward and store it in the DataFrame as two new columns

In [17]:
lats = []
longs = []
geolocator2 = Nominatim(user_agent = "ward-explorer")
for w in range(533):
    print(w)
    count = 0
    while count < 15:
        try:
            ward_address = london_df["Ward"][w] + ", London"
            co = geolocator2.geocode(ward_address)
            lats.append(co.latitude)
            longs.append(co.longitude)
            break
        except:
            count += 1
            continue
    if len(lats) != (w+1):
        lats.append(np.nan)
        longs.append(np.nan)
    print(lats[-1], longs[-1])
print("Done")

0
51.487621 0.1140504
1
51.5081402 -0.2732607
2
51.3586365 -0.0316347
3
51.3796916 -0.0742821
4
51.4361034 0.1259205
5
nan nan
6
51.5142477 -0.0757186
7
51.5124367 -0.1187414
8
51.5377679 -0.2979241
9
51.41284785 -0.0653006032639425
10
51.5319458 -0.1061056
11
51.3166288 0.0327167
12
51.5654371 -0.1349977
13
51.5841911 0.2209904
14
51.6455827 -0.236258
15
51.6228975 -0.1268518
16
51.4428285 -0.1514426
17
51.5125884 -0.3904739
18
51.5201501 -0.0986832
19
51.5389922 0.0804245
20
51.5858181 0.0886245
21
51.4626874 0.1617566
22
51.4718962 -0.2387441
23
51.4532994 0.1920241
24
51.6418265 -0.2429855
25
nan nan
26
51.5389351 -0.114735
27
51.4707933 -0.172214
28
51.5124135 -0.1876323
29
51.4070938 -0.0303183
30
51.5160797 0.0594257
31
51.5492654 0.1275378
32
51.5610299 0.1478793
33
51.3719875 -0.1323928
34
51.49805155 -0.255735576874332
35
51.4982128 -0.1534998
36
51.4310809 -0.0245145
37
51.3437847 -0.2011522
38
51.3437847 -0.2011522
39
51.5473926 -0.1683005
40
51.4894735 0.1473593
41
51.4970

In [18]:
london_df["Latitude"] = lats
london_df["Longitude"] = longs

In [19]:
london_df

,Ward,Borough,Latitude,Longitude
0,Abbey Wood,Bexley Greenwich,51.487621,0.114050
1,Acton,Ealing Hammersmith and Fulham,51.508140,-0.273261
2,Addington,Croydon,51.358637,-0.031635
3,Addiscombe,Croydon,51.379692,-0.074282
4,Albany Park,Bexley,51.436103,0.125920
5,Aldborough Hatch,Redbridge,NaN,NaN
6,Aldgate,City,51.514248,-0.075719
7,Aldwych,Westminster,51.512437,-0.118741
8,Alperton,Brent,51.537768,-0.297924
9,Anerley,Bromley,51.412848,-0.065301


#### Now we must deal with the missing coordinates

#### We will first find out which neighbourhoods do not have coordinates in the df.

In [20]:
london_df[london_df.isna().any(axis=1)]

,Ward,Borough,Latitude,Longitude
5,Aldborough Hatch,Redbridge,NaN,NaN
25,"Barnet (also Chipping Barnet, High Barnet)",Barnet,NaN,NaN
44,"Bexley (also Old Bexley, Bexley Village)",Bexley,NaN,NaN
45,Bexleyheath (also Bexley New Town),Bexley,NaN,NaN
66,Bromley (also Bromley-by-Bow),Tower Hamlets,NaN,NaN
166,Enfield Wash,Enfield,NaN,NaN
302,Marylebone (also St Marylebone),Westminster,NaN,NaN
454,"Sydenham (also Lower Sydenham, Upper Sydenham)",Lewisham Bromley,NaN,NaN
517,Widmore (also Widmore Green),Bromley,NaN,NaN


#### I will update these by hand, as they are a small amount. The column index 2 stands for Latitude and column index 3 stands for Longitude.

In [21]:
london_df.iloc[5, 2] = 51.575331032
london_df.iloc[5, 3] = 0.100999596
london_df.iloc[25, 2] = 51.64
london_df.iloc[25, 3] = -0.194
london_df.iloc[34, 2] = 51.4833
london_df.iloc[34, 3] = -0.2667
london_df.iloc[44, 2] = 51.4333
london_df.iloc[44, 3] = 0.15
london_df.iloc[45, 2] = 51.46
london_df.iloc[45, 3] = 0.16
#london_df.iloc[51, 2] = 51.476852
#london_df.iloc[51, 3] = -0.000500
london_df.iloc[66, 2] = 51.3333
london_df.iloc[66, 3] = 0.0833
"""
london_df.iloc[69, 2] = 51.5407
london_df.iloc[69, 3] = -0.2103
london_df.iloc[84, 2] = 51.444641
london_df.iloc[84, 3] = -0.020106
london_df.iloc[99, 2] = 51.483498066
london_df.iloc[99, 3] = -0.255165646
london_df.iloc[114, 2] = 51.5144
london_df.iloc[114, 3] = -0.1225
london_df.iloc[128, 2] = 51.4833
london_df.iloc[128, 3] = 0.0
london_df.iloc[143, 2] = 51.48666472
london_df.iloc[143, 3] = -0.191999232
"""
london_df.iloc[166, 2] = 51.669797
london_df.iloc[166, 3] = -0.040981
london_df.iloc[302, 2] = 51.518875
london_df.iloc[302, 3] = -0.149895
london_df.iloc[454, 2] = 51.4329
london_df.iloc[454, 3] = -0.0690
london_df.iloc[517, 2] = 51.405306
london_df.iloc[517, 3] = 0.035033

#### We should now see no rows with missing values.

In [22]:
london_df[london_df.isna().any(axis=1)]

,Ward,Borough,Latitude,Longitude


#### We will now drop the Borough column and store the final df in a csv file to avoid going through this process again.

In [24]:
london_df.drop("Borough", axis=1, inplace=True)
london_df.head()

,Ward,Latitude,Longitude
0,Abbey Wood,51.487621,0.114050
1,Acton,51.508140,-0.273261
2,Addington,51.358637,-0.031635
3,Addiscombe,51.379692,-0.074282
4,Albany Park,51.436103,0.125920


In [25]:
project.save_data(data=london_df.to_csv(index=False), file_name="ward_locations.csv", overwrite=True)

NameError: name 'project' is not defined

### I will now obtain the data regarding income.

In [61]:
income_url = "https://data.london.gov.uk/download/household-income-estimates-small-areas/23a759da-0526-4a27-970b-817196e11c6f/modelled-household-income-estimates-wards.csv"

In [73]:
income_df = pd.read_csv(income_url, encoding="latin1")
income_df.head()

,Code,Ward name,LAD code,Borough,Mean 2001/02,Mean 2002/03,Mean 2003/04,Mean 2004/05,Mean 2005/06,Mean 2006/07,...,Median 2003/04,Median 2004/05,Median 2005/06,Median 2006/07,Median 2007/08,Median 2008/09,Median 2009/10,Median 2010/11,Median 2011/12,Median 2012/13
0,E09000001,City of London,E09000001,City of London,"£65,120","£66,410","£69,800","£74,510","£76,200","£81,300",...,"£45,150","£48,070","£48,380","£51,410","£53,110","£54,960","£57,520","£59,240","£62,290","£63,620"
1,E05000026,Abbey,E09000002,Barking and Dagenham,"£25,540","£25,920","£27,150","£28,920","£29,550","£31,530",...,"£23,810","£25,320","£25,490","£27,100","£28,030","£29,050","£30,470","£31,450","£33,140","£33,920"
2,E05000027,Alibon,E09000002,Barking and Dagenham,"£23,980","£24,390","£25,610","£27,350","£28,000","£29,940",...,"£22,360","£23,840","£24,050","£25,640","£26,580","£27,610","£29,010","£30,000","£31,670","£32,470"
3,E05000028,Becontree,E09000002,Barking and Dagenham,"£25,250","£25,580","£26,750","£28,450","£29,030","£30,920",...,"£23,440","£24,880","£25,000","£26,540","£27,420","£28,390","£29,730","£30,660","£32,270","£33,000"
4,E05000029,Chadwell Heath,E09000002,Barking and Dagenham,"£25,950","£26,290","£27,490","£29,240","£29,830","£31,770",...,"£24,120","£25,610","£25,730","£27,320","£28,220","£29,200","£30,580","£31,520","£33,170","£33,920"


#### I will drop the columns that are irrelevant to my study

In [74]:
income_df.drop(["Code", "LAD code"], inplace=True, axis=1)
income_df.head()

,Ward name,Borough,Mean 2001/02,Mean 2002/03,Mean 2003/04,Mean 2004/05,Mean 2005/06,Mean 2006/07,Mean 2007/08,Mean 2008/09,...,Median 2003/04,Median 2004/05,Median 2005/06,Median 2006/07,Median 2007/08,Median 2008/09,Median 2009/10,Median 2010/11,Median 2011/12,Median 2012/13
0,City of London,City of London,"£65,120","£66,410","£69,800","£74,510","£76,200","£81,300","£83,050","£89,340",...,"£45,150","£48,070","£48,380","£51,410","£53,110","£54,960","£57,520","£59,240","£62,290","£63,620"
1,Abbey,Barking and Dagenham,"£25,540","£25,920","£27,150","£28,920","£29,550","£31,530","£32,230","£34,710",...,"£23,810","£25,320","£25,490","£27,100","£28,030","£29,050","£30,470","£31,450","£33,140","£33,920"
2,Alibon,Barking and Dagenham,"£23,980","£24,390","£25,610","£27,350","£28,000","£29,940","£30,660","£33,080",...,"£22,360","£23,840","£24,050","£25,640","£26,580","£27,610","£29,010","£30,000","£31,670","£32,470"
3,Becontree,Barking and Dagenham,"£25,250","£25,580","£26,750","£28,450","£29,030","£30,920","£31,560","£33,950",...,"£23,440","£24,880","£25,000","£26,540","£27,420","£28,390","£29,730","£30,660","£32,270","£33,000"
4,Chadwell Heath,Barking and Dagenham,"£25,950","£26,290","£27,490","£29,240","£29,830","£31,770","£32,430","£34,870",...,"£24,120","£25,610","£25,730","£27,320","£28,220","£29,200","£30,580","£31,520","£33,170","£33,920"


#### I will rename the Ward name column to enforce data integrity when the tables are merged and have the same data in both tables recognised as the same variable.

In [75]:
income_df.rename(columns={"Ward name": "Ward"}, inplace=True)
income_df.head()

,Ward,Borough,Mean 2001/02,Mean 2002/03,Mean 2003/04,Mean 2004/05,Mean 2005/06,Mean 2006/07,Mean 2007/08,Mean 2008/09,...,Median 2003/04,Median 2004/05,Median 2005/06,Median 2006/07,Median 2007/08,Median 2008/09,Median 2009/10,Median 2010/11,Median 2011/12,Median 2012/13
0,City of London,City of London,"£65,120","£66,410","£69,800","£74,510","£76,200","£81,300","£83,050","£89,340",...,"£45,150","£48,070","£48,380","£51,410","£53,110","£54,960","£57,520","£59,240","£62,290","£63,620"
1,Abbey,Barking and Dagenham,"£25,540","£25,920","£27,150","£28,920","£29,550","£31,530","£32,230","£34,710",...,"£23,810","£25,320","£25,490","£27,100","£28,030","£29,050","£30,470","£31,450","£33,140","£33,920"
2,Alibon,Barking and Dagenham,"£23,980","£24,390","£25,610","£27,350","£28,000","£29,940","£30,660","£33,080",...,"£22,360","£23,840","£24,050","£25,640","£26,580","£27,610","£29,010","£30,000","£31,670","£32,470"
3,Becontree,Barking and Dagenham,"£25,250","£25,580","£26,750","£28,450","£29,030","£30,920","£31,560","£33,950",...,"£23,440","£24,880","£25,000","£26,540","£27,420","£28,390","£29,730","£30,660","£32,270","£33,000"
4,Chadwell Heath,Barking and Dagenham,"£25,950","£26,290","£27,490","£29,240","£29,830","£31,770","£32,430","£34,870",...,"£24,120","£25,610","£25,730","£27,320","£28,220","£29,200","£30,580","£31,520","£33,170","£33,920"


In [76]:
london_income_df = pd.merge(london_df, income_df)
london_income_df.head()

,Ward,Borough,Mean 2001/02,Mean 2002/03,Mean 2003/04,Mean 2004/05,Mean 2005/06,Mean 2006/07,Mean 2007/08,Mean 2008/09,...,Median 2003/04,Median 2004/05,Median 2005/06,Median 2006/07,Median 2007/08,Median 2008/09,Median 2009/10,Median 2010/11,Median 2011/12,Median 2012/13
0,Abbey Wood,Greenwich,"£25,800","£26,210","£27,480","£29,310","£29,970","£32,010","£32,740","£35,290",...,"£21,970","£23,390","£23,570","£25,080","£25,970","£26,940","£28,270","£29,200","£30,790","£31,540"
1,Addiscombe,Croydon,"£32,950","£33,240","£34,610","£36,620","£37,160","£39,370","£39,950","£42,720",...,"£28,430","£30,040","£30,030","£31,710","£32,570","£33,520","£34,910","£35,780","£37,450","£38,070"
2,Alperton,Brent,"£28,030","£28,120","£29,120","£30,690","£31,030","£32,780","£33,190","£35,440",...,"£23,770","£25,000","£24,900","£26,200","£26,850","£27,580","£28,690","£29,380","£30,740","£31,250"
3,Balham,Wandsworth,"£50,720","£51,610","£54,140","£57,690","£58,910","£62,780","£64,040","£68,820",...,"£38,260","£40,670","£40,880","£43,390","£44,780","£46,290","£48,410","£49,820","£52,340","£53,420"
4,Barkingside,Redbridge,"£34,470","£34,770","£36,200","£38,320","£38,900","£41,230","£41,870","£44,820",...,"£29,370","£31,040","£31,040","£32,790","£33,710","£34,720","£36,190","£37,140","£38,920","£39,620"


#### We shall now drop the Borough column and make the Ward column become the index to identify the row, as it is unique and more descriptive than numerical indices.